In [4]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import scipy
import psycopg2
import tensorflow as ts
from collections import defaultdict

con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['figure.facecolor'] = 'white'

In [5]:
from sqlalchemy import create_engine
engine = create_engine('postgres://%s@localhost/%s'%('Joy', 'codeforces'))

# create Y values

In [6]:
# note this is 4x faster than getting it from sql
df_smooth = pd.read_csv('user_ratings_smoothed.csv', engine = 'c')

## calculate difference
Only need to run this once
```
gusr = df_smooth.groupby('handle')
stack = []

for usr, dfu in gusr:
    dfu.is_copy=False
    dfu.sort_values('ratingupdatetimeseconds', inplace=True)
    stack.append(dfu)

df_smooth = pd.concat(stack)
for month in range(1, 6):
    curr = df_smooth["smoothed_%dmonths" % month]
    prev = np.roll(curr, 1)

    delta = curr - prev
    df_smooth["delta_smoothed_%dmonths" % month] = delta

df_smooth.head(50)

## output to sql and csv

df_smooth.to_csv('user_ratings_smoothed.csv', index=False, header=True)

df_smooth.to_sql('user_rating_smooth', engine, if_exists='replace')
```

# Features
 **problem type**
 * contest
 * virtual
 * etc
 
**problem info**
 * tags
 * rating
 * point value
 
**submission info**
 * number of wrong answers
 * number of TLE
 * number of compile errors
 * time between first submission and solve
 * relative time to competition
 
**user info**
 * current smooth rating
 * volatility?
 * lag can be estimated from user rating and smoothed rating, but do we want it??

## Getting distinct values for categorial variables

In [7]:
cur.execute("SELECT * FROM handles")
all_handles = [h[0] for h in cur.fetchall()]

Run this code **once only** to get a list of keys. This takes ~20 minutes

```
q = """
SELECT DISTINCT handle, contestid, problemid FROM submissions;
"""
cur.execute(q)
keys = cur.fetchall()

with open('handle_cid_pid_keys.txt', 'w') as f:
    for k in keys:
        f.write(','.join(k) + '\n')
```

In [8]:
with open('handle_cid_pid_keys.txt') as f:
    keys = [line.strip() for line in f.readlines()]

### Get all tags
Only need to run this once:
```
cur.execute("""
SELECT DISTINCT tag FROM tags
""")
all_tags = [t[0] for t in cur.fetchall()]

df_all_tags = pd.DataFrame(all_tags)
df_all_tags.rename_axis({0: 'tag'}, axis=1, inplace=True)
df_all_tags.to_sql('all_tags', engine, if_exists='replace')
```

In [9]:
cur.execute("SELECT tag FROM all_tags")
all_tags = set([t[0] for t in cur.fetchall()])

### Get distinct verdicts

Only need to run this once:
```
cur.execute("""
SELECT DISTINCT verdict FROM submissions
""")
all_verdicts = [v[0] for v in cur.fetchall()]

df_all_verdicts = pd.DataFrame(all_verdicts)
df_all_verdicts.rename_axis({0: 'verdict'}, axis=1, inplace=True)
df_all_verdicts.to_sql('all_verdicts', engine, if_exists='replace')
```

In [10]:
cur.execute("SELECT verdict FROM all_verdicts")
all_verdicts = set([t[0] for t in cur.fetchall()])

### Get distinct participant types
```
cur.execute("""
SELECT DISTINCT participanttype FROM submissions
""")
all_participanttypes = [v[0] for v in cur.fetchall()]

df_all_participanttypes = pd.DataFrame(all_participanttypes)
df_all_participanttypes.rename_axis({0: 'participanttype'}, axis=1, inplace=True)
df_all_participanttypes.to_sql('all_participanttypes', engine, if_exists='replace')
```

In [11]:
cur.execute("SELECT participanttype FROM all_participanttypes")
all_participanttypes = set([t[0] for t in cur.fetchall()])

### programming languages
```
cur.execute("""
SELECT DISTINCT language FROM submissions
""")
all_language = [v[0] for v in cur.fetchall()]

df_all_language = pd.DataFrame(all_language)
df_all_language.rename_axis({0: 'language'}, axis=1, inplace=True)
df_all_language.to_sql('all_language', engine, if_exists='replace')
```

In [16]:
cur.execute("SELECT language FROM all_language")
all_language = set([t[0] for t in cur.fetchall()])

## problem stats

### problem rating and tags

In [12]:
df_prate = pd.read_sql("SELECT * FROM problem_rating", con)
df_prate.set_index(['contestid', 'problemid'], inplace=True)

df_tags = pd.read_sql("SELECT * FROM tags", con)
df_tags.set_index(['contestid', 'problemid'], inplace=True)

df_smooth.reset_index(inplace=True)
df_smooth.set_index(['handle'], inplace=True)
df_smooth.drop('contestname', axis=1, inplace=True)
df_smooth.drop('time', axis=1, inplace=True)

In [13]:
user_dict = defaultdict(list)
keys = [k.split(',') for k in keys]
for k in keys:
    user_dict[ k[0] ].append(k)

In [21]:
from os.path import exists
cnt = 0
#user = '-----'

def getTraining(user):
    #filename = 'train_rnn.csv'
    filename = 'rnn_train/%s.csv' % user
    print filename
    trainlist = []
    ur = df_smooth.loc[user]
    #ur.drop(['contestid', 'rank'], inplace=True, axis=1)
    ur.is_copy = False
    ur.reset_index(inplace=True)
    
    for k in user_dict[user]:
        q = """
        SELECT * FROM submissions
            WHERE
                handle = '%s'
                AND
                contestid = '%s'
                AND
                problemid = '%s'
        """ % (k[0], k[1], k[2])

        df = pd.read_sql(q, con)
        df.is_copy = False

        ex = dict()
        
        # generic problem info
        ex['points'] = df.points[0]
        ex['starttimeseconds'] = min(df.starttimeseconds)
        ex['stoptimeseconds'] = max(df.starttimeseconds)
        
        # user rating info ----------------------------------
        # find closest next contest
        # if there is no next contest,then skip this entry
        idx = ur.ratingupdatetimeseconds >= ex['stoptimeseconds']
        if not np.any(idx):
            continue
        tur = ur.loc[idx]
        tur.is_copy = False
        idx = tur.ratingupdatetimeseconds == min(tur.ratingupdatetimeseconds)
        tur = tur.loc[idx].to_dict(orient='records')[0]
        ex.update(tur)


        # verdicts
        vcnt = df.verdict.value_counts()
        vdict = vcnt.to_dict()
        ex.update(vdict)

        # participant type
        pcnt = df.participanttype.value_counts()
        pdict = pcnt.to_dict()
        for t in pdict.iterkeys():
            ex[t] = 1

        # language
        lcnt = df.language.value_counts()
        ldict = lcnt.to_dict()
        ex.update(ldict)

        # problem rating
        if (k[1], k[2]) in df_prate:
            ex['problem_rating'] = df_prate.loc[k[1],k[2]].values[0]
        else:
            ex['problem_rating'] = -1

        # time to solves
        solvetime = df.loc[df.verdict=='OK', 'starttimeseconds']
        if len(solvetime) > 0:
            ex['solvetimeseconds'] = min(solvetime)
        else:
            ex['solvetimeseconds'] = -1

        trainlist.append(ex)

    #    cnt += 1
    #    if cnt > 10:
    #        break

    df_train = pd.DataFrame.from_dict(trainlist)
    for t in all_tags:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_verdicts:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_participanttypes:
        if t not in df_train.columns:
            df_train[t] = np.nan
    for t in all_language:
        if t not in df_train.columns:
            df_train[t] = np.nan
            
#    if df_train.shape[1] != 100:
#        print df_train.columns

#    if exists(filename):
    df_train.to_csv(filename, mode='a', index=False, header=True)
#    else:
#        df_train.to_csv(filename, mode='a', index=False, header=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 429


In [22]:
lastidx = 0
for i, user in enumerate(all_handles[lastidx:]):
    print lastidx + i, user
    getTraining(user)

0 tourist
rnn_train/tourist.csv
1 LHiC
rnn_train/LHiC.csv
2 Petr
rnn_train/Petr.csv
3 ainta
rnn_train/ainta.csv
4 Um_nik
rnn_train/Um_nik.csv
5 W4yneb0t
rnn_train/W4yneb0t.csv
6 RomaWhite
rnn_train/RomaWhite.csv
7 moejy0viiiiiv
rnn_train/moejy0viiiiiv.csv
8 I_love_Tanya_Romanova
rnn_train/I_love_Tanya_Romanova.csv
9 Zlobober
rnn_train/Zlobober.csv
10 -XraY-
rnn_train/-XraY-.csv
11 izrak
rnn_train/izrak.csv
12 vepifanov
rnn_train/vepifanov.csv
13 TakanashiRikka
rnn_train/TakanashiRikka.csv
14 bmerry
rnn_train/bmerry.csv
15 al13n
rnn_train/al13n.csv
16 Merkurev
rnn_train/Merkurev.csv
17 TankEngineer
rnn_train/TankEngineer.csv
18 Shik
rnn_train/Shik.csv
19 V--o_o--V
rnn_train/V--o_o--V.csv
20 y0105w49
rnn_train/y0105w49.csv
21 jiry_2
rnn_train/jiry_2.csv
22 zemen
rnn_train/zemen.csv
23 cubelover
rnn_train/cubelover.csv
24 cki86201
rnn_train/cki86201.csv
25 matthew99
rnn_train/matthew99.csv
26 scott_wu
rnn_train/scott_wu.csv
27 Alex_2oo8
rnn_train/Alex_2oo8.csv
28 xyz111
rnn_train/xyz111.c

231 chokudai
rnn_train/chokudai.csv
232 AngryBacon
rnn_train/AngryBacon.csv
233 zimpha
rnn_train/zimpha.csv
234 bayleef
rnn_train/bayleef.csv
235 KFDong
rnn_train/KFDong.csv
236 pavel.savchenkov
rnn_train/pavel.savchenkov.csv
237 witua
rnn_train/witua.csv
238 wjmsbmr
rnn_train/wjmsbmr.csv
239 platypus179
rnn_train/platypus179.csv
240 BudAlNik
rnn_train/BudAlNik.csv
241 knightL
rnn_train/knightL.csv
242 wrong
rnn_train/wrong.csv
243 cchao
rnn_train/cchao.csv
244 Romka
rnn_train/Romka.csv
245 xiaowuc1
rnn_train/xiaowuc1.csv
246 chffy
rnn_train/chffy.csv
247 johnasselta
rnn_train/johnasselta.csv
248 Dmitry_Egorov
rnn_train/Dmitry_Egorov.csv
249 SergeyRogulenko
rnn_train/SergeyRogulenko.csv
250 Shapo
rnn_train/Shapo.csv
251 PORONGON
rnn_train/PORONGON.csv
252 JOHNKRAM
rnn_train/JOHNKRAM.csv
253 pekempey
rnn_train/pekempey.csv
254 IlyaLos
rnn_train/IlyaLos.csv
255 piob
rnn_train/piob.csv
256 Gassa
rnn_train/Gassa.csv
257 rng_50216
rnn_train/rng_50216.csv
258 shy_love_myf
rnn_train/shy_love_

458 david942j
rnn_train/david942j.csv
459 sy2006
rnn_train/sy2006.csv
460 AmberFrame
rnn_train/AmberFrame.csv
461 aropan
rnn_train/aropan.csv
462 curs0r
rnn_train/curs0r.csv
463 glassices
rnn_train/glassices.csv
464 ichyo
rnn_train/ichyo.csv
465 lythanh
rnn_train/lythanh.csv
466 marX
rnn_train/marX.csv
467 _0w0_
rnn_train/_0w0_.csv
468 blue.boy
rnn_train/blue.boy.csv
469 gvaibhav21
rnn_train/gvaibhav21.csv
470 rankalee
rnn_train/rankalee.csv
471 zerokugi
rnn_train/zerokugi.csv
472 Dylans
rnn_train/Dylans.csv
473 Stigius
rnn_train/Stigius.csv
474 Taube
rnn_train/Taube.csv
475 bjin
rnn_train/bjin.csv
476 maryanna2015
rnn_train/maryanna2015.csv
477 2222
rnn_train/2222.csv
478 sfiction
rnn_train/sfiction.csv
479 yubowenok
rnn_train/yubowenok.csv
480 Paohu
rnn_train/Paohu.csv
481 sexyprincess91
rnn_train/sexyprincess91.csv
482 I_love_Margot_Robbie
rnn_train/I_love_Margot_Robbie.csv
483 IbukiSuika
rnn_train/IbukiSuika.csv
484 White_Bear
rnn_train/White_Bear.csv
485 Temirulan
rnn_train/Temiru

687 azizkhan
rnn_train/azizkhan.csv
688 mrNobody
rnn_train/mrNobody.csv
689 Chelly
rnn_train/Chelly.csv
690 Smylic
rnn_train/Smylic.csv
691 lydrainbowcat
rnn_train/lydrainbowcat.csv
692 JustN
rnn_train/JustN.csv
693 Solaris
rnn_train/Solaris.csv
694 alecsyde
rnn_train/alecsyde.csv
695 kliu31415
rnn_train/kliu31415.csv
696 senek_k
rnn_train/senek_k.csv
697 xorfire
rnn_train/xorfire.csv
698 Neroysq
rnn_train/Neroysq.csv
699 abyssmall
rnn_train/abyssmall.csv
700 razia_sultana
rnn_train/razia_sultana.csv
701 ufo
rnn_train/ufo.csv
702 williamhu08
rnn_train/williamhu08.csv
703 rdfz_yjy
rnn_train/rdfz_yjy.csv
704 k21
rnn_train/k21.csv
705 rudradevbasak
rnn_train/rudradevbasak.csv
706 ch_egor
rnn_train/ch_egor.csv
707 FreeMoneyCity
rnn_train/FreeMoneyCity.csv
708 Gomhog
rnn_train/Gomhog.csv
709 Life_is_like_a_boat
rnn_train/Life_is_like_a_boat.csv
710 MikhailRubinchik
rnn_train/MikhailRubinchik.csv
711 Oleg_Smirnov
rnn_train/Oleg_Smirnov.csv
712 jiangshibiao
rnn_train/jiangshibiao.csv
713 Demb

913 gog.gerard
rnn_train/gog.gerard.csv
914 master_miu
rnn_train/master_miu.csv
915 zerotrac
rnn_train/zerotrac.csv
916 Sisu
rnn_train/Sisu.csv
917 anton
rnn_train/anton.csv
918 hellman_
rnn_train/hellman_.csv
919 ivan.popelyshev
rnn_train/ivan.popelyshev.csv
920 kevinsogo
rnn_train/kevinsogo.csv
921 mgch
rnn_train/mgch.csv
922 quuuu
rnn_train/quuuu.csv
923 AlexanderBolshakov
rnn_train/AlexanderBolshakov.csv
924 Kissshot
rnn_train/Kissshot.csv
925 i_wanna_no_exams_forever
rnn_train/i_wanna_no_exams_forever.csv
926 romanandreev
rnn_train/romanandreev.csv
927 Skird
rnn_train/Skird.csv
928 no_motivation
rnn_train/no_motivation.csv
929 temich
rnn_train/temich.csv
930 tooweak
rnn_train/tooweak.csv
931 ditoly
rnn_train/ditoly.csv
932 fenzhang
rnn_train/fenzhang.csv
933 DozingBear
rnn_train/DozingBear.csv
934 Salvare000
rnn_train/Salvare000.csv
935 ccdd4211
rnn_train/ccdd4211.csv
936 james007
rnn_train/james007.csv
937 matrix
rnn_train/matrix.csv
938 netkuba
rnn_train/netkuba.csv
939 netman
r

...

4193 shettynamit
rnn_train/shettynamit.csv
4194 thanksgiving
rnn_train/thanksgiving.csv
4195 world2
rnn_train/world2.csv
4196 00001
rnn_train/00001.csv
4197 1403mashaonan
rnn_train/1403mashaonan.csv
4198 Affina
rnn_train/Affina.csv
4199 AlwaysWrong
rnn_train/AlwaysWrong.csv
4200 ConanKudo
rnn_train/ConanKudo.csv
4201 Erich
rnn_train/Erich.csv
4202 Harmonica
rnn_train/Harmonica.csv
4203 KeiyaSakabe
rnn_train/KeiyaSakabe.csv
4204 KenjiH
rnn_train/KenjiH.csv
4205 LSY_is_retired_
rnn_train/LSY_is_retired_.csv
4206 LiuRunkY
rnn_train/LiuRunkY.csv
4207 M_W
rnn_train/M_W.csv
4208 Mohammad.Sj
rnn_train/Mohammad.Sj.csv
4209 RNS2
rnn_train/RNS2.csv
4210 The-Legend
rnn_train/The-Legend.csv
4211 Yudee
rnn_train/Yudee.csv
4212 Yukimai
rnn_train/Yukimai.csv
4213 abhay26
rnn_train/abhay26.csv
4214 arthuryang
rnn_train/arthuryang.csv
4215 bahuangliuhe
rnn_train/bahuangliuhe.csv
4216 dlhham
rnn_train/dlhham.csv
4217 gotop
rnn_train/gotop.csv
4218 ihave33cm
rnn_train/ihave33cm.csv
4219 maggot092
rnn_tra

4409 zck921031
rnn_train/zck921031.csv
4410 zlqiszlq
rnn_train/zlqiszlq.csv
4411 6eJIa9IzZzTeHb
rnn_train/6eJIa9IzZzTeHb.csv
4412 771491749
rnn_train/771491749.csv
4413 Anarivu
rnn_train/Anarivu.csv
4414 Andrekk
rnn_train/Andrekk.csv
4415 Hennkka
rnn_train/Hennkka.csv
4416 Hinidu
rnn_train/Hinidu.csv
4417 I_hate_my_color
rnn_train/I_hate_my_color.csv
4418 KirillB
rnn_train/KirillB.csv
4419 Lukasz16a
rnn_train/Lukasz16a.csv
4420 NEU20133823
rnn_train/NEU20133823.csv
4421 RFlare
rnn_train/RFlare.csv
4422 Sonechko
rnn_train/Sonechko.csv
4423 ThanhTran
rnn_train/ThanhTran.csv
4424 ZYQN
rnn_train/ZYQN.csv
4425 aangairbender
rnn_train/aangairbender.csv
4426 cjldl
rnn_train/cjldl.csv
4427 filix
rnn_train/filix.csv
4428 findingLoveOfMyLife
rnn_train/findingLoveOfMyLife.csv
4429 ivankrut856
rnn_train/ivankrut856.csv
4430 jackal_1586
rnn_train/jackal_1586.csv
4431 ktesar
rnn_train/ktesar.csv
4432 lemonsqueeze
rnn_train/lemonsqueeze.csv
4433 poao899
rnn_train/poao899.csv
4434 qscqesze_1
rnn_train

4626 stonebuddha
rnn_train/stonebuddha.csv
4627 vout
rnn_train/vout.csv
4628 zeppelin
rnn_train/zeppelin.csv
4629 zyf940104357
rnn_train/zyf940104357.csv
4630 Allen.fly
rnn_train/Allen.fly.csv
4631 Aphanasiy
rnn_train/Aphanasiy.csv
4632 Ashishgup
rnn_train/Ashishgup.csv
4633 BIT_silence
rnn_train/BIT_silence.csv
4634 Chortos-2
rnn_train/Chortos-2.csv
4635 DXHPeter
rnn_train/DXHPeter.csv
4636 FISHTOBY
rnn_train/FISHTOBY.csv
4637 HITLJR
rnn_train/HITLJR.csv
4638 Reyiz
rnn_train/Reyiz.csv
4639 abcd1927
rnn_train/abcd1927.csv
4640 diametralis
rnn_train/diametralis.csv
4641 gstsclq
rnn_train/gstsclq.csv
4642 hanxueshui
rnn_train/hanxueshui.csv
4643 inasnuha
rnn_train/inasnuha.csv
4644 innok96
rnn_train/innok96.csv
4645 jinshuwang
rnn_train/jinshuwang.csv
4646 konmaj
rnn_train/konmaj.csv
4647 misis
rnn_train/misis.csv
4648 nghiand
rnn_train/nghiand.csv
4649 pandazcy2
rnn_train/pandazcy2.csv
4650 pichulia
rnn_train/pichulia.csv
4651 sroyal
rnn_train/sroyal.csv
4652 totoroXD
rnn_train/totoroXD

4844 rotoZOOM
rnn_train/rotoZOOM.csv
4845 shym_red
rnn_train/shym_red.csv
4846 vmrajas
rnn_train/vmrajas.csv
4847 whitestone
rnn_train/whitestone.csv
4848 wittyskull
rnn_train/wittyskull.csv
4849 xiaoziqian
rnn_train/xiaoziqian.csv
4850 yml
rnn_train/yml.csv
4851 API
rnn_train/API.csv
4852 Belgabad.
rnn_train/Belgabad..csv
4853 Dixtosa
rnn_train/Dixtosa.csv
4854 DorMOUSE-None
rnn_train/DorMOUSE-None.csv
4855 Goofy57
rnn_train/Goofy57.csv
4856 Jeannette
rnn_train/Jeannette.csv
4857 LFLONG
rnn_train/LFLONG.csv
4858 Manu_Zhu
rnn_train/Manu_Zhu.csv
4859 Miyamizu_Mitsuha
rnn_train/Miyamizu_Mitsuha.csv
4860 Nakochi
rnn_train/Nakochi.csv
4861 Nordrassil
rnn_train/Nordrassil.csv
4862 Sherbina_Evgeniy
rnn_train/Sherbina_Evgeniy.csv
4863 SpeedCuber
rnn_train/SpeedCuber.csv
4864 Zymphony77
rnn_train/Zymphony77.csv
4865 arknave
rnn_train/arknave.csv
4866 chenwenxiao
rnn_train/chenwenxiao.csv
4867 i_love_netman
rnn_train/i_love_netman.csv
4868 kammola
rnn_train/kammola.csv
4869 leave_me_alone
rnn_t

TypeError: Cannot reset_index inplace on a Series to create a DataFrame